In [63]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import scipy.stats as st
import numpy as np
# linear modelling
import statsmodels.api as sm
from scipy.stats import linregress
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.ensemble import RandomForestRegressor
# metrics
from sklearn.metrics import mean_squared_error
import seaborn as sns

# Census API Key
from census import Census
from config import api_key
c = Census(api_key, year=2017)

In [2]:
df=pd.read_csv("Airbnb_Texas_Rentals.csv")
df

,Unnamed: 0,average_rate_per_night,bedrooms_count,city,date_of_listing,description,latitude,longitude,title,url
0,1,$27,2,Humble,May 2016,Welcome to stay in private room with queen bed...,30.020138,-95.293996,2 Private rooms/bathroom 10min from IAH airport,https://www.airbnb.com/rooms/18520444?location...
1,2,$149,4,San Antonio,November 2010,"Stylish, fully remodeled home in upscale NW – ...",29.503068,-98.447688,Unique Location! Alamo Heights - Designer Insp...,https://www.airbnb.com/rooms/17481455?location...
2,3,$59,1,Houston,January 2017,'River house on island close to the city' \nA ...,29.829352,-95.081549,River house near the city,https://www.airbnb.com/rooms/16926307?location...
3,4,$60,1,Bryan,February 2016,Private bedroom in a cute little home situated...,30.637304,-96.337846,Private Room Close to Campus,https://www.airbnb.com/rooms/11839729?location...
4,5,$75,2,Fort Worth,February 2017,Welcome to our original 1920's home. We recent...,32.747097,-97.286434,The Porch,https://www.airbnb.com/rooms/17325114?location...
...,...,...,...,...,...,...,...,...,...,...
18254,18255,$60,1,Dallas,March 2013,An entire 1 bedroom 700+sqft condo in the hear...,32.892303,-96.772049,Quiet comfort living in Dallas,https://www.airbnb.com/rooms/1011576?location=...
18255,18256,$99,2,San Antonio,June 2015,An inviting 1920's cottage home in a popular u...,29.452893,-98.486756,Midtown Cottage Near Riverwalk,https://www.airbnb.com/rooms/18766940?location...
18256,18257,$13,1,Dallas,December 2016,Amazing 3BHK Apartment in a picturesque Commun...,33.001955,-96.777615,Room in Dallas!,https://www.airbnb.com/rooms/18719059?location...
18257,18258,$65,2,San Antonio,October 2016,My quaint and cozy home is conveniently locate...,29.450142,-98.505333,Vibrant Spacious Loft!,https://www.airbnb.com/rooms/18179329?location...


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18259 entries, 0 to 18258
Data columns (total 10 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Unnamed: 0              18259 non-null  int64  
 1   average_rate_per_night  18231 non-null  object 
 2   bedrooms_count          18256 non-null  object 
 3   city                    18259 non-null  object 
 4   date_of_listing         18259 non-null  object 
 5   description             18257 non-null  object 
 6   latitude                18225 non-null  float64
 7   longitude               18225 non-null  float64
 8   title                   18256 non-null  object 
 9   url                     18259 non-null  object 
dtypes: float64(2), int64(1), object(7)
memory usage: 1.4+ MB


In [4]:
df.count()

Unnamed: 0                18259
average_rate_per_night    18231
bedrooms_count            18256
city                      18259
date_of_listing           18259
description               18257
latitude                  18225
longitude                 18225
title                     18256
url                       18259
dtype: int64

In [5]:
clean_df = df.dropna()
clean_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 18217 entries, 0 to 18258
Data columns (total 10 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Unnamed: 0              18217 non-null  int64  
 1   average_rate_per_night  18217 non-null  object 
 2   bedrooms_count          18217 non-null  object 
 3   city                    18217 non-null  object 
 4   date_of_listing         18217 non-null  object 
 5   description             18217 non-null  object 
 6   latitude                18217 non-null  float64
 7   longitude               18217 non-null  float64
 8   title                   18217 non-null  object 
 9   url                     18217 non-null  object 
dtypes: float64(2), int64(1), object(7)
memory usage: 1.5+ MB


In [6]:
# summary data frame
clean_df2 = clean_df.loc[:, ["city", "date_of_listing", "bedrooms_count", "average_rate_per_night", "latitude", "longitude"]]
clean_df2

,city,date_of_listing,bedrooms_count,average_rate_per_night,latitude,longitude
0,Humble,May 2016,2,$27,30.020138,-95.293996
1,San Antonio,November 2010,4,$149,29.503068,-98.447688
2,Houston,January 2017,1,$59,29.829352,-95.081549
3,Bryan,February 2016,1,$60,30.637304,-96.337846
4,Fort Worth,February 2017,2,$75,32.747097,-97.286434
...,...,...,...,...,...,...
18254,Dallas,March 2013,1,$60,32.892303,-96.772049
18255,San Antonio,June 2015,2,$99,29.452893,-98.486756
18256,Dallas,December 2016,1,$13,33.001955,-96.777615
18257,San Antonio,October 2016,2,$65,29.450142,-98.505333


In [7]:
clean_df2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 18217 entries, 0 to 18258
Data columns (total 6 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   city                    18217 non-null  object 
 1   date_of_listing         18217 non-null  object 
 2   bedrooms_count          18217 non-null  object 
 3   average_rate_per_night  18217 non-null  object 
 4   latitude                18217 non-null  float64
 5   longitude               18217 non-null  float64
dtypes: float64(2), object(4)
memory usage: 996.2+ KB


In [8]:
clean_df2["date_of_listing"]=pd.to_datetime(clean_df2["date_of_listing"])

clean_df2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 18217 entries, 0 to 18258
Data columns (total 6 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   city                    18217 non-null  object        
 1   date_of_listing         18217 non-null  datetime64[ns]
 2   bedrooms_count          18217 non-null  object        
 3   average_rate_per_night  18217 non-null  object        
 4   latitude                18217 non-null  float64       
 5   longitude               18217 non-null  float64       
dtypes: datetime64[ns](1), float64(2), object(3)
memory usage: 996.2+ KB


In [9]:
clean_df2["year"]=[x.year for x in clean_df2.date_of_listing]
clean_df2.head()

,city,date_of_listing,bedrooms_count,average_rate_per_night,latitude,longitude,year
0,Humble,2016-05-01,2,$27,30.020138,-95.293996,2016
1,San Antonio,2010-11-01,4,$149,29.503068,-98.447688,2010
2,Houston,2017-01-01,1,$59,29.829352,-95.081549,2017
3,Bryan,2016-02-01,1,$60,30.637304,-96.337846,2016
4,Fort Worth,2017-02-01,2,$75,32.747097,-97.286434,2017


In [10]:
unique_year = clean_df2["year"].nunique()
unique_year

10

In [11]:
clean_df2["month"]=[x.month for x in clean_df2.date_of_listing]
clean_df2.head()

,city,date_of_listing,bedrooms_count,average_rate_per_night,latitude,longitude,year,month
0,Humble,2016-05-01,2,$27,30.020138,-95.293996,2016,5
1,San Antonio,2010-11-01,4,$149,29.503068,-98.447688,2010,11
2,Houston,2017-01-01,1,$59,29.829352,-95.081549,2017,1
3,Bryan,2016-02-01,1,$60,30.637304,-96.337846,2016,2
4,Fort Worth,2017-02-01,2,$75,32.747097,-97.286434,2017,2


In [12]:
clean_df2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 18217 entries, 0 to 18258
Data columns (total 8 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   city                    18217 non-null  object        
 1   date_of_listing         18217 non-null  datetime64[ns]
 2   bedrooms_count          18217 non-null  object        
 3   average_rate_per_night  18217 non-null  object        
 4   latitude                18217 non-null  float64       
 5   longitude               18217 non-null  float64       
 6   year                    18217 non-null  int64         
 7   month                   18217 non-null  int64         
dtypes: datetime64[ns](1), float64(2), int64(2), object(3)
memory usage: 1.3+ MB


In [13]:
mask1 = clean_df2.loc[clean_df2['year']==2012].index
mask2 = clean_df2.loc[clean_df2['year']==2011].index
mask3 = clean_df2.loc[clean_df2['year']==2010].index
mask4 = clean_df2.loc[clean_df2['year']==2009].index
mask5 = clean_df2.loc[clean_df2['year']==2008].index

In [14]:
mask1.array

<PandasArray>
[   12,    37,    89,   222,   262,   276,   311,   327,   340,   351,
 ...
 18090, 18120, 18136, 18137, 18146, 18159, 18184, 18205, 18213, 18215]
Length: 799, dtype: int64

In [15]:
clean_df3 = clean_df2[~(clean_df2.year.isin([2012, 2011, 2010, 2009, 2008]))].reset_index()
clean_df3.head()

,index,city,date_of_listing,bedrooms_count,average_rate_per_night,latitude,longitude,year,month
0,0,Humble,2016-05-01,2,$27,30.020138,-95.293996,2016,5
1,2,Houston,2017-01-01,1,$59,29.829352,-95.081549,2017,1
2,3,Bryan,2016-02-01,1,$60,30.637304,-96.337846,2016,2
3,4,Fort Worth,2017-02-01,2,$75,32.747097,-97.286434,2017,2
4,5,Conroe,2016-08-01,4,$250,30.370455,-95.385319,2016,8


In [16]:
clean_df3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16799 entries, 0 to 16798
Data columns (total 9 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   index                   16799 non-null  int64         
 1   city                    16799 non-null  object        
 2   date_of_listing         16799 non-null  datetime64[ns]
 3   bedrooms_count          16799 non-null  object        
 4   average_rate_per_night  16799 non-null  object        
 5   latitude                16799 non-null  float64       
 6   longitude               16799 non-null  float64       
 7   year                    16799 non-null  int64         
 8   month                   16799 non-null  int64         
dtypes: datetime64[ns](1), float64(2), int64(3), object(3)
memory usage: 1.2+ MB


In [17]:
# Create the bins in which Data will be held
# Bins are  12-1-2016, 7-1-2016, 1-1-2018
bins = [0, 3, 6, 9, 12]

# Create the names for the five bins
group_names = ["First", "Second", "Third", "Fourth"]

In [18]:
clean_df3["Date_Quarter"] = pd.cut(clean_df3["month"], bins, labels=group_names, include_lowest=True)
clean_df3

,index,city,date_of_listing,bedrooms_count,average_rate_per_night,latitude,longitude,year,month,Date_Quarter
0,0,Humble,2016-05-01,2,$27,30.020138,-95.293996,2016,5,Second
1,2,Houston,2017-01-01,1,$59,29.829352,-95.081549,2017,1,First
2,3,Bryan,2016-02-01,1,$60,30.637304,-96.337846,2016,2,First
3,4,Fort Worth,2017-02-01,2,$75,32.747097,-97.286434,2017,2,First
4,5,Conroe,2016-08-01,4,$250,30.370455,-95.385319,2016,8,Third
...,...,...,...,...,...,...,...,...,...,...
16794,18254,Dallas,2013-03-01,1,$60,32.892303,-96.772049,2013,3,First
16795,18255,San Antonio,2015-06-01,2,$99,29.452893,-98.486756,2015,6,Second
16796,18256,Dallas,2016-12-01,1,$13,33.001955,-96.777615,2016,12,Fourth
16797,18257,San Antonio,2016-10-01,2,$65,29.450142,-98.505333,2016,10,Fourth


In [19]:
# summary data frame
clean_df4 = clean_df3.loc[:, ["city", "year", "Date_Quarter", "bedrooms_count", "average_rate_per_night", "latitude", "longitude"]]
clean_df4.head()

,city,year,Date_Quarter,bedrooms_count,average_rate_per_night,latitude,longitude
0,Humble,2016,Second,2,$27,30.020138,-95.293996
1,Houston,2017,First,1,$59,29.829352,-95.081549
2,Bryan,2016,First,1,$60,30.637304,-96.337846
3,Fort Worth,2017,First,2,$75,32.747097,-97.286434
4,Conroe,2016,Third,4,$250,30.370455,-95.385319


In [20]:
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="http")

In [21]:
location = geolocator.reverse("30.020138, -95.293996")
location.raw

{'place_id': 291315935,
 'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. https://osm.org/copyright',
 'osm_type': 'way',
 'osm_id': 198797061,
 'lat': '30.02013976623853',
 'lon': '-95.29412005281664',
 'display_name': '20521, Kenswick Drive, Bordersville, Harris County, Texas, 77338, United States',
 'address': {'house_number': '20521',
  'road': 'Kenswick Drive',
  'hamlet': 'Bordersville',
  'county': 'Harris County',
  'state': 'Texas',
  'postcode': '77338',
  'country': 'United States',
  'country_code': 'us'},
 'boundingbox': ['30.020089766239',
  '30.020189766239',
  '-95.294170052817',
  '-95.294070052817']}

In [ ]:
counties = []
zipcodes = []

for indx,row in clean_df4.iterrows():
    lat=row["latitude"]
    lon=row["longitude"]
    location = geolocator.reverse(f"{lat}, {lon}", timeout=None)
    
    county = location.raw["address"].get("county")
    zipcode = location.raw["address"].get("postcode")
    counties.append(county)
    zipcodes.append(zipcode)

    if indx % 100==0:
        print(indx)
    

In [60]:
len(counties)

32

In [ ]:
clean_df4.head()


In [61]:
len(zipcodes)

32

In [ ]:
counties

In [ ]:
clean_df4['counties'] = counties
clean_df4['zipcodes'] = zipcodes
clean_df4

In [ ]:
# Export file as a CSV, without the Pandas index, but with the header - So we don't have to run the above code every
# time we open the file. 
clean_df4.to_csv("airbnb_counties.csv", index=False, header=True)

In [23]:
# read in airbnb_counties csv
airbnb_df=pd.read_csv("airbnb_counties.csv")
airbnb_df

,city,year,Date_Quarter,bedrooms_count,average_rate_per_night,latitude,longitude,counties,zipcodes
0,Humble,2016,Second,2,$27,30.020138,-95.293996,Harris County,77338
1,Houston,2017,First,1,$59,29.829352,-95.081549,Harris County,77049
2,Bryan,2016,First,1,$60,30.637304,-96.337846,Brazos County,77802
3,Fort Worth,2017,First,2,$75,32.747097,-97.286434,Tarrant County,76103
4,Conroe,2016,Third,4,$250,30.370455,-95.385319,Montgomery County,77303
...,...,...,...,...,...,...,...,...,...
16794,Dallas,2013,First,1,$60,32.892303,-96.772049,Dallas County,75230
16795,San Antonio,2015,Second,2,$99,29.452893,-98.486756,Bexar County,78212
16796,Dallas,2016,Fourth,1,$13,33.001955,-96.777615,Collin County,75252
16797,San Antonio,2016,Fourth,2,$65,29.450142,-98.505333,Bexar County,78212


In [59]:
airbnb_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16799 entries, 0 to 16798
Data columns (total 9 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   city                    16799 non-null  object 
 1   year                    16799 non-null  int64  
 2   Date_Quarter            16799 non-null  object 
 3   bedrooms_count          16799 non-null  object 
 4   average_rate_per_night  16799 non-null  object 
 5   latitude                16799 non-null  float64
 6   longitude               16799 non-null  float64
 7   counties                16799 non-null  object 
 8   zipcodes                16030 non-null  object 
dtypes: float64(2), int64(1), object(6)
memory usage: 1.2+ MB


In [57]:
# read in census csv
census_df=pd.read_csv("acs2017_county_data.csv")
census_df.head()

,CountyId,State,County,TotalPop,Men,Women,Hispanic,White,Black,Native,...,Walk,OtherTransp,WorkAtHome,MeanCommute,Employed,PrivateWork,PublicWork,SelfEmployed,FamilyWork,Unemployment
0,1001,Alabama,Autauga County,55036,26899,28137,2.7,75.4,18.9,0.3,...,0.6,1.3,2.5,25.8,24112,74.1,20.2,5.6,0.1,5.2
1,1003,Alabama,Baldwin County,203360,99527,103833,4.4,83.1,9.5,0.8,...,0.8,1.1,5.6,27.0,89527,80.7,12.9,6.3,0.1,5.5
2,1005,Alabama,Barbour County,26201,13976,12225,4.2,45.7,47.8,0.2,...,2.2,1.7,1.3,23.4,8878,74.1,19.1,6.5,0.3,12.4
3,1007,Alabama,Bibb County,22580,12251,10329,2.4,74.6,22.0,0.4,...,0.3,1.7,1.5,30.0,8171,76.0,17.4,6.3,0.3,8.2
4,1009,Alabama,Blount County,57667,28490,29177,9.0,87.4,1.5,0.3,...,0.4,0.4,2.1,35.0,21380,83.9,11.9,4.0,0.1,4.9


In [35]:
census_tx = census_df.loc[census_df.State == "Texas"].reset_index()
census_tx.head()

,index,CountyId,State,County,TotalPop,Men,Women,Hispanic,White,Black,...,Walk,OtherTransp,WorkAtHome,MeanCommute,Employed,PrivateWork,PublicWork,SelfEmployed,FamilyWork,Unemployment
0,2523,48001,Texas,Anderson County,57747,35292,22455,17.3,59.7,20.9,...,0.5,0.7,3.6,23.7,19102,73.2,20.4,6.2,0.2,4.3
1,2524,48003,Texas,Andrews County,17577,8980,8597,55.4,41.1,1.5,...,0.0,0.8,2.5,19.8,8054,85.3,8.7,5.7,0.3,4.5
2,2525,48005,Texas,Angelina County,87700,42707,44993,21.5,61.2,14.7,...,1.7,0.9,2.2,18.7,36164,77.2,16.7,6.0,0.1,7.6
3,2526,48007,Texas,Aransas County,24832,12448,12384,27.0,67.7,1.6,...,2.8,0.5,6.8,20.8,10387,72.9,12.3,14.6,0.2,6.7
4,2527,48009,Texas,Archer County,8793,4335,4458,8.3,88.6,0.8,...,1.0,1.1,4.7,20.4,4344,75.6,14.5,9.8,0.0,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
249,2772,48499,Texas,Wood County,43315,21523,21792,9.6,83.2,5.5,...,2.5,1.0,5.6,30.2,15960,78.4,14.1,7.1,0.4,7.8
250,2773,48501,Texas,Yoakum County,8481,4286,4195,65.0,33.2,0.0,...,3.7,0.2,1.3,16.2,3755,71.7,17.5,10.7,0.1,7.5
251,2774,48503,Texas,Young County,18166,8922,9244,18.1,78.8,1.5,...,0.6,1.4,6.1,15.3,8248,69.8,17.1,11.9,1.2,4.5
252,2775,48505,Texas,Zapata County,14415,7190,7225,94.1,5.2,0.0,...,4.4,0.6,2.0,18.2,5146,66.0,21.0,12.6,0.4,12.4


In [36]:
census_tx.loc[census_tx.County == "Harris County"]

,index,CountyId,State,County,TotalPop,Men,Women,Hispanic,White,Black,...,Walk,OtherTransp,WorkAtHome,MeanCommute,Employed,PrivateWork,PublicWork,SelfEmployed,FamilyWork,Unemployment
100,2623,48201,Texas,Harris County,4525519,2251060,2274459,42.2,30.6,18.5,...,1.5,1.9,3.7,28.9,2180392,83.4,9.8,6.6,0.2,6.4


In [37]:
# summary data frame
clean_tx = census_tx.loc[:, ["County", "TotalPop", "Income", "Poverty"]]
clean_tx.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 254 entries, 0 to 253
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   County    254 non-null    object 
 1   TotalPop  254 non-null    int64  
 2   Income    254 non-null    int64  
 3   Poverty   254 non-null    float64
dtypes: float64(1), int64(2), object(1)
memory usage: 8.1+ KB


In [114]:
clean_tx = clean_tx.rename(columns = {"County":"counties"})
clean_tx.head()

,counties,TotalPop,Income,Poverty
0,Anderson County,57747,42313,15.7
1,Andrews County,17577,70753,11.9
2,Angelina County,87700,46472,19.1
3,Aransas County,24832,44601,17.8
4,Archer County,8793,63192,8.8


In [115]:
clean_tx.loc[census_tx.County == "Harris County"]

,counties,TotalPop,Income,Poverty
100,Harris County,4525519,57791,16.8


In [116]:
census_data_airbnbtx = pd.merge(airbnb_df, clean_tx, on="counties")
census_data_airbnbtx.head()

,city,year,Date_Quarter,bedrooms_count,average_rate_per_night,latitude,longitude,counties,zipcodes,TotalPop,Income,Poverty
0,Humble,2016,Second,2,$27,30.020138,-95.293996,Harris County,77338,4525519,57791,16.8
1,Houston,2017,First,1,$59,29.829352,-95.081549,Harris County,77049,4525519,57791,16.8
2,Houston,2017,First,4,$1500,29.954680,-95.176070,Harris County,77044,4525519,57791,16.8
3,Katy,2017,First,4,$2999,29.823802,-95.730637,Harris County,77449,4525519,57791,16.8
4,Katy,2016,Third,3,$165,29.766553,-95.783192,Harris County,77494,4525519,57791,16.8


In [117]:
census_data_airbnbtx.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16799 entries, 0 to 16798
Data columns (total 12 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   city                    16799 non-null  object 
 1   year                    16799 non-null  int64  
 2   Date_Quarter            16799 non-null  object 
 3   bedrooms_count          16799 non-null  object 
 4   average_rate_per_night  16799 non-null  object 
 5   latitude                16799 non-null  float64
 6   longitude               16799 non-null  float64
 7   counties                16799 non-null  object 
 8   zipcodes                16030 non-null  object 
 9   TotalPop                16799 non-null  int64  
 10  Income                  16799 non-null  int64  
 11  Poverty                 16799 non-null  float64
dtypes: float64(3), int64(3), object(6)
memory usage: 1.7+ MB


In [118]:
corrs = census_data_airbnbtx.corr()
corrs

,year,latitude,longitude,TotalPop,Income,Poverty
year,1.000000,-0.051052,0.069336,-0.015643,-0.006607,0.013146
latitude,-0.051052,1.000000,-0.103368,0.076818,0.270531,-0.324150
longitude,0.069336,-0.103368,1.000000,0.387658,0.135663,0.132776
TotalPop,-0.015643,0.076818,0.387658,1.000000,-0.075115,0.160188
Income,-0.006607,0.270531,0.135663,-0.075115,1.000000,-0.830596
Poverty,0.013146,-0.324150,0.132776,0.160188,-0.830596,1.000000
